1. Descarga de datos
Descarga los datasets para vinos tintos y blancos desde la siguiente URL: 
Asegúrate de que ambos datasets se encuentren disponibles en tu entorno de trabajo como archivos CSV.
Idealmente, intenta llevarlo a cabo de manera programática para no tener que hacer operaciones manualmente.


In [1]:
#!pip install requests
#!pip install zipfile
#!pip install wget

In [2]:
# Librerias

import wget
import os
import zipfile
from os import remove
import pandas as pd

In [3]:
'Creo la carpeta csv'
ruta_csv='csv/'
if not os.path.exists(ruta_csv):
    os.mkdir(ruta_csv)

'Descargo el zip '
filezip = 'wine+quality.zip'
if not os.path.exists(filezip):
    download_http = "http://archive.ics.uci.edu/static/public/186/wine+quality.zip"
    try:
        wget.download(download_http)
    except:
        print('Error de conexion con la web')

'Descomprimo el zip en la ruta csv/'
file_name_src_zip = ""
if os.path.exists(filezip):
    archivozip= zipfile.ZipFile(filezip, mode="r")
    file_name_src_zip =archivozip.namelist()
    try:
        archivozip.extractall(ruta_csv)
    except:
        pass
    archivozip.close()
    remove(filezip)

In [4]:

'Listo en el directorio csv/ los ficheros'
ruta_actual=os.listdir(ruta_csv)
listname_csv=[]

'Identifico los ficheros con extension csv'
try:
    for x in ruta_actual:
        if x.split(".")[1]=='csv':
            listname_csv.append(x)

    'Asigno cada fichero csv a una variable'
    red=''
    white=''
    if 'red' in listname_csv[0]:
        red=ruta_csv+listname_csv[0]
    else:
        red=ruta_csv+listname_csv[1]
    if 'white' in listname_csv[0]:
        white=ruta_csv+listname_csv[0]
    else:
        white=ruta_csv+listname_csv[1]
except:
    print("Error no existen ficheros csv")

2. Combinar los datos

Usa Pandas para cargar ambos datasets en dataframes en memoria.

In [5]:
'Leo cada csv sabiendo que se separan por ; y le añado la nueva columna tipo'
df_1 = pd.read_csv(red, delimiter=';')
df_1['Type']='red'
df_2 = pd.read_csv(white,delimiter=';')
df_2['Type']='white'

In [6]:
df_1.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type
1298,5.7,0.600,0.00,1.4,0.063,11.0,18.0,0.99191,3.45,0.56,12.2,6,red
1533,8.0,0.390,0.30,1.9,0.074,32.0,84.0,0.99717,3.39,0.61,9.0,5,red
79,8.3,0.625,0.20,1.5,0.080,27.0,119.0,0.99720,3.16,1.12,9.1,4,red
1094,6.6,0.725,0.09,5.5,0.117,9.0,17.0,0.99655,3.35,0.49,10.8,6,red
945,10.2,0.440,0.42,2.0,0.071,7.0,20.0,0.99566,3.14,0.79,11.1,7,red


In [7]:
df_2.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type
1273,8.1,0.50,0.47,1.1,0.037,23.0,126.0,0.99380,3.21,0.42,10.9,5,white
2503,8.6,0.34,0.36,1.4,0.045,11.0,119.0,0.99556,3.17,0.47,9.4,4,white
3589,6.8,0.19,0.71,17.5,0.042,21.0,114.0,0.99784,2.85,0.50,9.5,6,white
3764,5.4,0.46,0.15,2.1,0.026,29.0,130.0,0.98953,3.39,0.77,13.4,8,white
1499,6.7,0.51,0.24,2.1,0.043,14.0,155.0,0.99040,3.22,0.60,13.0,6,white


Combina los datos en un único dataframe añadiendo una columna adicional que indique el tipo de vino (red o white). (debemos hacer un concat)

In [127]:
'Combino los 2 dataframe'

winequality=pd.concat([df_2, df_1]).reset_index(drop=True)

¿Cuántos registros tenemos?

In [9]:
filas, columnas = winequality.shape
f"El dataframe tiene {filas} filas y {columnas} columnas"

'El dataframe tiene 6497 filas y 13 columnas'

¿Cuántas variables y de qué tipo?

In [10]:
'¿Cuántas variables y de qué tipo?'
winequality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  Type                  6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


3. Filtrar atípicos y manejar datos ausentes (ANALISIS CON PANDAS)


•	Realiza un análisis estadístico o inspección visual de cada columna numérica para identificar valores atípicos.


In [39]:
winequality.describe()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [40]:
winequality.shape

(6497, 13)

In [91]:
winequality

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,white
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,white
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,red
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,red
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,red
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,red


In [167]:
#Obtengo todas la columnas menos la de tipo string

q1 = winequality.iloc[:,:-1].quantile(0.25)
q3 = winequality.iloc[:,:-1].quantile(0.75)
iqr = q3 - q1

atipicos = (winequality.iloc[:,:-1] < (q1 - 1.5 * iqr)) | (winequality.iloc[:,:-1] > (q3 + 1.5 * iqr))
atipicos

#Devuelve las filas atipicas
rows_atipicas = atipicos.any(axis=1)


#Serie con los índices de las filas atipicas 
s_atipicas = winequality.index[rows_atipicas]


#borrado de las filas a través del índice de las filas atipicas y reset del index del df
winequality_clean=winequality.drop(index=s_atipicas).reset_index(drop=True)


In [178]:
winequality_clean

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type
0,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,white
1,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
2,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
4,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,red
4836,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,red
4837,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,red
4838,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,red


4. Almacenar los datos limpios en SQLite


In [179]:
import sqlite3
from sqlalchemy import create_engine


•	Usa SQLite para almacenar el dataframe limpio en una base de datos persistente.


•	Sigue la documentación oficial de SQLite: SQLite Python Documentation.

In [182]:
try:

    engine = create_engine('sqlite:///winequality01.db', echo=True)
    sqlite_connection = engine.connect()

    sqlite_table = 'winequality_clean'
    winequality_clean.to_sql(sqlite_table, sqlite_connection, if_exists='fail')
    sqlite_connection.close()
except:
    print("error al crear la base de datos")

2025-07-08 19:38:24,557 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-08 19:38:24,561 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("winequality_clean")
2025-07-08 19:38:24,562 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-08 19:38:24,565 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("winequality_clean")
2025-07-08 19:38:24,565 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-08 19:38:24,568 INFO sqlalchemy.engine.Engine 
CREATE TABLE winequality_clean (
	"index" BIGINT, 
	"fixed acidity" FLOAT, 
	"volatile acidity" FLOAT, 
	"citric acid" FLOAT, 
	"residual sugar" FLOAT, 
	chlorides FLOAT, 
	"free sulfur dioxide" FLOAT, 
	"total sulfur dioxide" FLOAT, 
	density FLOAT, 
	"pH" FLOAT, 
	sulphates FLOAT, 
	alcohol FLOAT, 
	quality BIGINT, 
	"Type" TEXT
)


2025-07-08 19:38:24,570 INFO sqlalchemy.engine.Engine [no key 0.00122s] ()
2025-07-08 19:38:24,576 INFO sqlalchemy.engine.Engine CREATE INDEX ix_winequality_clean_index ON winequality_clean ("index")
2025-07

In [184]:
con = sqlite3.connect("winequality01.db")
cur = con.cursor()

In [189]:

for row in cur.execute("SELECT * FROM winequality_clean"):
    print(row)

(0, 6.3, 0.3, 0.34, 1.6, 0.049, 14.0, 132.0, 0.994, 3.3, 0.49, 9.5, 6, 'white')
(1, 8.1, 0.28, 0.4, 6.9, 0.05, 30.0, 97.0, 0.9951, 3.26, 0.44, 10.1, 6, 'white')
(2, 7.2, 0.23, 0.32, 8.5, 0.058, 47.0, 186.0, 0.9956, 3.19, 0.4, 9.9, 6, 'white')
(3, 7.2, 0.23, 0.32, 8.5, 0.058, 47.0, 186.0, 0.9956, 3.19, 0.4, 9.9, 6, 'white')
(4, 8.1, 0.28, 0.4, 6.9, 0.05, 30.0, 97.0, 0.9951, 3.26, 0.44, 10.1, 6, 'white')
(5, 6.2, 0.32, 0.16, 7.0, 0.045, 30.0, 136.0, 0.9949, 3.18, 0.47, 9.6, 6, 'white')
(6, 6.3, 0.3, 0.34, 1.6, 0.049, 14.0, 132.0, 0.994, 3.3, 0.49, 9.5, 6, 'white')
(7, 8.1, 0.22, 0.43, 1.5, 0.044, 28.0, 129.0, 0.9938, 3.22, 0.45, 11.0, 6, 'white')
(8, 8.1, 0.27, 0.41, 1.45, 0.033, 11.0, 63.0, 0.9908, 2.99, 0.56, 12.0, 5, 'white')
(9, 8.6, 0.23, 0.4, 4.2, 0.035, 17.0, 109.0, 0.9947, 3.14, 0.53, 9.7, 5, 'white')
(10, 7.9, 0.18, 0.37, 1.2, 0.04, 16.0, 75.0, 0.992, 3.18, 0.63, 10.8, 5, 'white')
(11, 6.6, 0.16, 0.4, 1.5, 0.044, 48.0, 143.0, 0.9912, 3.54, 0.52, 12.4, 7, 'white')
(12, 6.6, 0.17,

5. Realizar 3 consultas en SQLite. Basándote en los datos y las columnas del dataset, realiza las siguientes consultas:
   

In [234]:
import pandas as pd
import sqlalchemy

db_name = "winequality01.db"
table_name = "winequality_clean"

engine = sqlalchemy.create_engine("sqlite:///%s" % db_name, execution_options={"sqlite_raw_colnames": True})
df_winequality = pd.read_sql_table(table_name, engine)

 Consulta 1: ¿Cuál es el promedio de calidad (quality) por tipo de vino (type)?


In [262]:

'Consulta 1: ¿Cuál es el promedio de calidad (quality) por tipo de vino (type)?'

#for row in cur.execute("SELECT avg(quality),type FROM winequality group by type"):
#    print(row)

cur.execute("SELECT avg(quality),type FROM winequality group by type")
cur.fetchall()

[(5.6360225140712945, 'red'), (5.87790935075541, 'white')]

In [236]:
'Consulta 1 Con Pandas'
(
    df_winequality
    .groupby('Type')
    .agg(
        Promedio_calidad=("quality", "mean") 
    )
)



,Promedio_calidad
Type,
red,5.656891
white,5.848244


Consulta 2: ¿Cuántos vinos tienen un nivel de alcohol superior a 10.5, agrupados por tipo?


In [257]:
'Consulta 2: ¿Cuántos vinos tienen un nivel de alcohol superior a 10.5, agrupados por tipo?'
for row in cur.execute("SELECT  type, count(alcohol) FROM winequality_clean WHERE alcohol > 10.5 group by type "):
    print(row)


('red', 257)
('white', 1902)


In [254]:
'Consulta 2 Con pandas'
(
    df_winequality
    .assign(
        alcohol_Mayor_10=lambda df: df['alcohol'] > 10.5
    )
    .groupby('Type')
    .agg(  
        n_alcohol_Mayor=("alcohol_Mayor_10", "count")
    )
    
)

,n_alcohol_Mayor
Type,
red,682
white,4158


Consulta 3: Obtén el conteo de vinos por nivel de acidez (fixed acidity) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15). v1

In [ ]:
'Consulta 3: Obtén el conteo de vinos por nivel de acidez (fixed acidity) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15).'

query="""SELECT Type,
                CASE 
                    WHEN `fixed acidity` > 0 AND `fixed acidity` <= 5  THEN 'Rango1'
                    WHEN `fixed acidity` >= 5 AND `fixed acidity` <= 10 THEN 'Rango2'
                    WHEN `fixed acidity` >= 10 AND `fixed acidity` <= 15 THEN 'Rango3'
                    WHEN `fixed acidity` >= 15 THEN  'Range4'
                END,
                count(`fixed acidity`)             
        FROM winequality
        GROUP BY
                CASE 
                    WHEN `fixed acidity` > 0 AND `fixed acidity` <= 5  THEN 'Rango1'
                    WHEN `fixed acidity` >= 5 AND `fixed acidity` <= 10 THEN 'Rango2'
                    WHEN `fixed acidity` >= 10 AND `fixed acidity` <= 15 THEN 'Rango3'
                    WHEN `fixed acidity` >= 15 THEN  'Range4'
                END 
        """

for row in cur.execute(query):
    print(row)

OperationalError: near "ASC": syntax error

Consulta 3: Obtén el conteo de vinos por nivel de acidez (fixed acidity) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15). v2

In [28]:
'Consulta 3 Con pandas'
'Obtén el conteo de vinos por nivel de acidez (fixed acidity) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15)'

rango1=df_winequality.loc[(df_winequality['fixed acidity'] > 0) & (df_winequality['fixed acidity'] < 5)]
rango2=df_winequality.loc[(df_winequality['fixed acidity'] > 5) & (df_winequality['fixed acidity'] < 10)]
rango3=df_winequality.loc[(df_winequality['fixed acidity'] > 10) & (df_winequality['fixed acidity'] < 15)]

rango1['']





KeyError: ''

6. Exportar datos a JSONLines -> De cara a una potencial insercion en una base de datos noSQL como mongoDB, podemos servirnos de pandas para preparar los datos.